# Finetune a T5 model to translate from English to French

Task Description: We will finetune the Google T5 transformer model to translate from English to French. The dataset used is a subset of the OPUS Books.

Original Tutorial: https://huggingface.co/docs/transformers/tasks/translation

In [ ]:
!pip install -qqq transformers datasets evaluate accelerate bitsandbytes loralib peft

# Load the OPUS Books Dataset

In [ ]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr", split = 'train[0:5000]')

In [ ]:
# Split the dataset into a train and test set
books = books.train_test_split(test_size=0.2)

In [ ]:
# Look at the data
import pprint
pprint.pprint(books['train'][1])

# The translation column is our model input


In [ ]:
# Preprocessing
## Load Model
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Make a PEFT Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig, TaskType


# Create a config corresponding to the PEFT method
peft_config = LoraConfig(
    task_type = TaskType.SEQ_2_SEQ_LM,
    inference_mode = False,
    #target_modules= [""],
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [ ]:
# Wrap base model
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model = get_peft_model(model, peft_config)


In [ ]:
print(model)

In [ ]:
model.print_trainable_parameters()

# Preprocessing

We need to create a preprocessing function that performs the following operations:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.

2. Tokenize the input (English) and target (French) separately because you can’t tokenize French text with a tokenizer pretrained on an English vocabulary.

3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

def preprocess_function(examples):
  inputs = [prefix + example[source_lang] for example in examples["translation"]]
  targets = [example[target_lang] for example in examples['translation']]
  model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
  return model_inputs

In [ ]:
# Operation to apply to every instance
tokenized_books = books.map(preprocess_function, batched=True)

In [ ]:
# Create a batch of examples, with dynamic padding. Use the appropriate collator function
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate

We want to create a `compute_metrics` function that monitors a metric during training. For this task, use the SacreBLEU metric.

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Train using the Trainer API
The main training steps are:

1. Define training hyperparameters using a model specific TrainingArguments function. At the end of each epoch, the Trainer will evaluate the defined loss metric and save the training checkpoint.

2. Pass the training arguments to a Trainer function alongside the model, dataset, tokenizer, data collator.

3. Call train() to finetune the model

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# Evaluate the fine tuned model and obtain the perplexity score
import math

eval_results = trainer.evaluate()
print(f"sacrebleu: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("t5_translator")

In [ ]:
# In this case, the tokenizer was not saved automatically, save it manually in the model folder for inference
tokenizer.save_pretrained("t5_translator", legacy_format=False)

# Inference

Use model for inference using a pipeline wrapper

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "t5_translator"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
device = "cuda"
model = model.to(device)
model.eval()

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [ ]:
# Inference Pipeline using Pytorch
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5_translator")
inputs = tokenizer(text, return_tensors="pt").input_ids

with torch.no_grad():
  outputs = model.generate(input_ids = inputs.to(device), max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)